In [1]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [2]:
class State(TypedDict):
    #Messafes have the type "list". The "add_messages" function in the annotation defines how this state key should be updated (in this case, it appends messages to the list, rather than overwriting them)
    messages:Annotated[list, add_messages]

graph_builder=StateGraph(State)

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
#way 1 to initialize llm
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D135346570>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D1367025D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
#way 2 to initialize llm
from langchain.chat_models import init_chat_model
llm = init_chat_model("llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000015678972D20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000015678973260>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
#Creating the Node for Chatbot
def chatbot(state:State):
    return { "messages": [ llm.invoke(state["messages"])]}

In [6]:
#Building stateGraph for Chatbot 
graph_builder=StateGraph(State)
#Adding Node to the graph
graph_builder.add_node("llmchatbot", chatbot)

#Adding Edge to the graph
graph_builder.add_edge(START, "llmchatbot")
graph_builder.add_edge("llmchatbot", END)

#Compiling the graph
graph = graph_builder.compile()


In [ ]:
#Run the Chatbot
response = graph.invoke({"messages":"Hii, Write ABCD... Fully till Z and then explain who you are why you are and how you are !"})
response["messages"][-1].content
response  #see the whole message object with proper annotated format data in it and how it is stored in the state

{'messages': [HumanMessage(content='Hii, Write ABCD... Fully till Z and then explain who you are why you are and how you are !', additional_kwargs={}, response_metadata={}, id='303631e3-fc47-4c3a-8a08-6edf2ecc79a5'),
  AIMessage(content="I'd be happy to write the alphabet from A to Z for you.\n\nA\nB\nC\nD\nE\nF\nG\nH\nI\nJ\nK\nL\nM\nN\nO\nP\nQ\nR\nS\nT\nU\nV\nW\nX\nY\nZ\n\nNow, let's talk about who I am and why I exist.\n\nI am a computer program designed to simulate conversation and answer questions to the best of my knowledge. My primary purpose is to assist and provide useful information to users like you. I'm a type of artificial intelligence (AI) called a natural language processing (NLP) model.\n\nI was created through a process called deep learning, which involves training large datasets of text to recognize patterns and relationships between words. This training enables me to understand the context and meaning behind language, allowing me to generate human-like responses.\n\nI